In [5]:
import time
import math

import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
!wget 'https://drive.google.com/uc?export=download&id=1nuMuI2dO0JXdU5MuZjGHADC1Y6jvanaw' -O new_skels.csv

--2021-07-08 19:06:21--  https://drive.google.com/uc?export=download&id=1nuMuI2dO0JXdU5MuZjGHADC1Y6jvanaw
Resolving drive.google.com (drive.google.com)... 172.217.2.110, 2607:f8b0:4004:82f::200e
Connecting to drive.google.com (drive.google.com)|172.217.2.110|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kb6elnck0cbdr66i4ri9g5gt357qmhl1/1625771175000/14273303898397132356/*/1nuMuI2dO0JXdU5MuZjGHADC1Y6jvanaw?e=download [following]
--2021-07-08 19:06:23--  https://doc-00-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kb6elnck0cbdr66i4ri9g5gt357qmhl1/1625771175000/14273303898397132356/*/1nuMuI2dO0JXdU5MuZjGHADC1Y6jvanaw?e=download
Resolving doc-00-58-docs.googleusercontent.com (doc-00-58-docs.googleusercontent.com)... 142.250.73.225, 2607:f8b0:4004:82a::2001
Connecting to doc-00-58-docs.googleusercontent.com (doc-00-58-docs.go

1. Сгенерировать меньший датасет из 8-10 классов движения

In [8]:
skeletons_df = pd.read_csv('/content/new_skels.csv')

In [9]:
skeletons_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3336,3337,3338,3339,3340,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350,3351,3352,3353,3354,3355,3356,3357,3358,3359,3360,3361,3362,3363,3364,3365,3366,3367,3368,3369,3370,3371,3372,3373,3374,labels
0,0.107169,0.155336,3.774813,0.087842,0.424236,3.710918,0.068912,0.688434,3.634974,0.095212,0.807942,3.603217,-0.030894,0.570796,3.575153,-0.054834,0.338406,3.564177,-0.023961,0.113836,3.579079,-0.003036,0.071249,3.582160,0.202670,0.593185,3.712649,0.265672,0.382403,3.773768,0.335933,0.218966,3.774634,0.348793,0.138982,3.772001,0.051786,0.148485,3.711796,0.009193,...,0.018002,0.173563,3.738454,0.003108,-0.120043,3.810589,-0.034193,-0.410203,3.955529,0.027257,-0.471157,3.989867,0.125138,0.180552,3.774267,0.123769,-0.101110,3.890709,0.077180,-0.422957,4.046787,0.114206,-0.478615,3.949024,0.059637,0.623877,3.668810,0.300735,0.539580,3.595297,0.275155,0.478890,3.580000,0.204374,0.591374,3.486948,0.265206,0.617021,3.454333,1
1,0.073530,0.176115,3.790838,0.066357,0.431843,3.724868,0.057691,0.688121,3.649235,0.063602,0.803959,3.605012,-0.051390,0.576677,3.577165,0.060506,0.373254,3.536352,0.233468,0.523937,3.537280,0.312053,0.623063,3.524348,0.210907,0.603342,3.685688,0.375831,0.598065,3.628843,0.237076,0.584001,3.498875,0.213082,0.581713,3.494584,0.017938,0.173724,3.738734,0.003253,...,0.039503,0.216764,3.724097,0.014837,-0.098823,3.804500,-0.001431,-0.460940,4.005896,0.033438,-0.532394,3.925732,0.147237,0.245336,3.764869,0.151130,-0.048351,3.856603,0.076178,-0.385269,4.068417,0.066847,-0.486819,4.028674,0.051623,0.600825,3.401490,0.051330,0.775686,3.147952,0.042513,0.782981,3.131667,0.184349,0.609286,3.721297,0.140844,0.632283,3.666000,1
2,0.101858,0.157313,3.801223,0.106297,0.410928,3.751010,0.110345,0.659381,3.690153,0.069111,0.729826,3.626038,0.007819,0.556308,3.768062,-0.011128,0.582839,3.525778,0.056220,0.643828,3.305092,0.035716,0.686819,3.120791,0.170006,0.596938,3.717140,0.251414,0.652381,3.494979,0.234761,0.585732,3.640310,0.224816,0.651398,3.693118,0.039176,0.153576,3.758578,0.014972,...,0.068534,0.136442,3.691469,-0.016502,-0.140213,3.826027,-0.020635,-0.406493,3.960613,0.033393,-0.480396,3.986403,0.169888,0.137194,3.737295,0.174022,-0.121408,3.647927,0.104815,-0.356079,3.965588,0.149731,-0.409237,3.870815,0.094339,0.623390,3.684305,0.123914,0.249691,3.581933,0.023094,0.287091,3.642000,0.188379,0.091022,3.504948,0.142348,0.125880,3.552154,1
3,-0.017126,-0.549648,2.910658,0.008213,-0.278351,2.923760,0.030900,-0.010747,2.911902,0.035279,0.112427,2.903560,-0.106856,-0.106927,2.904222,-0.111414,-0.303022,2.865315,-0.030457,-0.430152,2.728033,-0.010259,-0.458600,2.678936,0.175180,-0.111796,2.903249,0.204783,-0.307510,2.886162,0.106341,-0.434892,2.747414,0.084417,-0.467307,2.701238,-0.082788,-0.537297,2.882745,-0.013058,...,-0.068641,-0.571650,2.915643,-0.078144,-0.494346,2.586306,-0.046216,-0.848132,2.589291,-0.048863,-0.900273,2.528113,0.046701,-0.560144,2.889013,0.116660,-0.373217,2.635323,-0.005860,-0.697047,2.511928,-0.028205,-0.775199,2.503626,-0.028977,-0.110459,2.851648,-0.283529,-0.468955,2.834120,-0.266852,-0.429082,2.857666,-0.044864,-0.524298,2.542837,-0.037103,-0.439468,2.542185,2
4,-0.012737,-0.566869,2.961922,-0.019491,-0.305559,2.894948,-0.032567,-0.047151,2.827832,-0.062442,0.069258,2.819594,-0.147424,-0.150081,2.844229,-0.194911,-0.226845,3.038487,-0.295766,-0.356791,2.983296,-0.295845,-0.407596,2.866120,0.106113,-0.101370,2.802265,0.095277,-0.301241,2.668104,0.027370,-0.437057,2.579191,0.001693,-0.478563,2.566986,-0.071199,-0.566213,2.942274,-0.085087,...,-0.044147,-0.508898,2.924855,-0.047596,-0.591335,2.588038,-0.051339,-0.854784,2.592133,-0.046010,-0.915295,2.515439,0.080394,-0.491245,2.918082,0.105395,-0.594797,2.588854,0.099358,-0.858555,2.572868,0.089104,-0.910380,2.466472,-0.018910,-0.110001,2.794079,-0.041126,-0.473166,2.507934,-0.067171,-0.438300,2.529875,0.031100,-0.445410,2.366435,0.067

In [10]:
skeletons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Columns: 3376 entries, 0 to labels
dtypes: float64(3375), int64(1)
memory usage: 31.0 MB


In [11]:
skeletons_df.shape

(1202, 3376)

In [12]:
skeletons_df.labels.value_counts()

2    123
6    121
9    120
8    120
7    120
5    120
4    120
3    120
1    120
0    118
Name: labels, dtype: int64

In [13]:
action_classes =  {7: 0, 15: 1, 17: 2, 3: 3, 5: 4, 9: 5, 19: 6, 13: 7, 1: 8, 11: 9}
    
LABELS = {
    1: 'drink water',
    3: 'brushing teeth',
    5: 'drop',
    7: 'throw',
    9: 'standing up (from sitting position)',
    11: 'reading',
    13: 'tear up paper',
    15: 'take off jacket',
    17: 'take off a shoe',
    19: 'take off glasses',
}

LABELS = {action_classes[k]:v for k,v in LABELS.items()}
LABELS

{0: 'throw',
 1: 'take off jacket',
 2: 'take off a shoe',
 3: 'brushing teeth',
 4: 'drop',
 5: 'standing up (from sitting position)',
 6: 'take off glasses',
 7: 'tear up paper',
 8: 'drink water',
 9: 'reading'}

2. Обучить уже существующую модель (предварительно проанализировав, какие параметры модели нужно изменить)

In [14]:
class LSTM_net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_num):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.lstm = nn.LSTM(input_dim, 
                            hidden_dim, 
                            layer_num, 
                            batch_first=True)
        
        self.dr = nn.Dropout2d(0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        lstm_out, (hn, cn) = self.lstm(x)
        
        out = self.fc(lstm_out[:, -1])
        
        return out

In [15]:
frames = 45

In [16]:
class SkeletonDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.labels = data.iloc[:, -1]
        self.data_len = len(self.data)
        
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx, :-1]).reshape(frames, 25 * 3)
        label = self.labels[idx]
        
        if self.transform:
            item = self.transform(item)
        
        return item, label

In [17]:
dataset = SkeletonDataset(skeletons_df)

In [23]:
train_len = int(0.75*len(dataset))

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset,
    [train_len, len(dataset) - train_len]
)
train_loader = DataLoader(
    train_dataset,
    batch_size=16, 
    shuffle=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=1, 
    shuffle=False
)

In [24]:
n_hidden = 128
n_joints = 25 * 3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(
    n_joints,
    n_hidden,
    n_categories,
    n_layer
)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [25]:
def class_from_output(output):
    top_n, top_i = output.topk(1)
    class_i = top_i[0].item()
    return LABELS[class_i], class_i


def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return f'{m}m {int(s)}s'

In [26]:
def train(model):
  criterion = nn.CrossEntropyLoss()
  learning_rate = 0.0007
  optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  all_losses = []
  start = time.time()
  counter = 0


  for epoch in range(150):  
      current_loss = 0
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          
          inputs, labels = data[0].to(device), data[1].to(device)
          optimizer.zero_grad()
      
          output = model(inputs.float())
          loss = criterion(output, labels)
          loss.backward()
          optimizer.step() 

          current_loss += loss.item()
          pred_label = LABELS[int(labels[0])]

          if counter % 500 == 0:
              guess, guess_i = class_from_output(output)
              correct = '✓' if guess == pred_label else f'✗ ({pred_label})'
              print(f'epoch : {epoch} iter : {i} ({time_since(start)})', end=' ')
              print(f'{loss:.4f}  / {guess} {correct}')

          
          counter = counter + 1
          
      if counter % 100 == 0:
          all_losses.append(current_loss / 25)
          current_loss = 0

In [27]:
n_hidden = 128
n_joints = 25 * 3
n_categories = len(LABELS)
n_layer = 2

rnn = LSTM_net(
    n_joints,
    n_hidden,
    n_categories,
    n_layer
)
rnn.to(device)

train(rnn)

epoch : 0 iter : 0 (0m 0s) 2.3266  / drop ✗ (take off glasses)
epoch : 8 iter : 44 (0m 16s) 2.2839  / drop ✗ (throw)
epoch : 17 iter : 31 (0m 32s) 2.2746  / drink water ✗ (throw)
epoch : 26 iter : 18 (0m 48s) 2.1940  / take off a shoe ✓
epoch : 35 iter : 5 (1m 4s) 2.2370  / drink water ✗ (take off a shoe)
epoch : 43 iter : 49 (1m 21s) 2.2343  / take off a shoe ✓
epoch : 52 iter : 36 (1m 37s) 2.1431  / drink water ✗ (reading)
epoch : 61 iter : 23 (1m 53s) 1.9214  / take off a shoe ✓
epoch : 70 iter : 10 (2m 9s) 2.2974  / drop ✗ (reading)
epoch : 78 iter : 54 (2m 26s) 2.0500  / take off a shoe ✗ (drop)
epoch : 87 iter : 41 (2m 42s) 2.0010  / standing up (from sitting position) ✓
epoch : 96 iter : 28 (2m 58s) 1.8940  / drink water ✓
epoch : 105 iter : 15 (3m 14s) 1.6701  / drink water ✗ (throw)
epoch : 114 iter : 2 (3m 31s) 1.8835  / drop ✓
epoch : 122 iter : 46 (3m 47s) 1.7551  / standing up (from sitting position) ✗ (throw)
epoch : 131 iter : 33 (4m 3s) 1.7493  / reading ✗ (drink water)

3. Изменить модель: посмотреть зависимость от количества LSTM модулей в модели

In [30]:
class LSTM_net_2(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_num):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.lstm = nn.LSTM(input_dim, 
                            hidden_dim, 
                            layer_num, 
                            batch_first=True)
        
        self.lstm_1 = nn.LSTM(hidden_dim, 
                          hidden_dim, 
                          layer_num, 
                          batch_first=True)
        
        self.dr = nn.Dropout2d(0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        x, (hn, cn) = self.lstm(x)

        x, (hn, cn) = self.lstm_1(x, (hn, cn))
        
        out = self.fc(x[:, -1])
        
        return out

In [31]:
n_hidden = 128
n_joints = 25 * 3
n_categories = len(LABELS)
n_layer = 2

rnn = LSTM_net_2(
    n_joints,
    n_hidden,
    n_categories,
    n_layer
)
rnn.to(device)

train(rnn)

epoch : 0 iter : 0 (0m 0s) 2.2736  / standing up (from sitting position) ✗ (drop)
epoch : 8 iter : 44 (0m 18s) 2.2990  / drop ✗ (standing up (from sitting position))
epoch : 17 iter : 31 (0m 36s) 2.3086  / drop ✗ (brushing teeth)
epoch : 26 iter : 18 (0m 54s) 2.2974  / drop ✗ (take off jacket)
epoch : 35 iter : 5 (1m 12s) 2.2992  / drop ✗ (standing up (from sitting position))
epoch : 43 iter : 49 (1m 30s) 2.2992  / drop ✓
epoch : 52 iter : 36 (1m 49s) 2.2807  / drop ✓
epoch : 61 iter : 23 (2m 7s) 2.3097  / drop ✗ (take off a shoe)
epoch : 70 iter : 10 (2m 25s) 2.2862  / drop ✗ (drink water)
epoch : 78 iter : 54 (2m 43s) 2.3084  / drop ✗ (take off a shoe)
epoch : 87 iter : 41 (3m 1s) 2.2813  / drop ✓
epoch : 96 iter : 28 (3m 20s) 2.2862  / drop ✗ (standing up (from sitting position))
epoch : 105 iter : 15 (3m 38s) 2.3014  / drop ✗ (tear up paper)
epoch : 114 iter : 2 (3m 56s) 2.2953  / drop ✗ (take off glasses)
epoch : 122 iter : 46 (4m 14s) 2.3046  / drop ✗ (take off glasses)
epoch : 1

In [32]:
class LSTM_net_3(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_num):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.lstm = nn.LSTM(input_dim, 
                            hidden_dim, 
                            layer_num, 
                            batch_first=True)
        
        self.lstm_1 = nn.LSTM(hidden_dim, 
                              hidden_dim, 
                              layer_num, 
                              batch_first=True)
        
        self.lstm_2 = nn.LSTM(hidden_dim, 
                              hidden_dim, 
                              layer_num, 
                              batch_first=True)
        
        self.dr = nn.Dropout2d(0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        x, (hn, cn) = self.lstm(x)

        x, (hn, cn) = self.lstm_1(x, (hn, cn))

        x, (hn, cn) = self.lstm_2(x, (hn, cn))

        out = self.fc(x[:, -1])
        
        return out

In [33]:
n_hidden = 128
n_joints = 25 * 3
n_categories = len(LABELS)
n_layer = 2

rnn = LSTM_net_3(
    n_joints,
    n_hidden,
    n_categories,
    n_layer
)
rnn.to(device)

train(rnn)

epoch : 0 iter : 0 (0m 0s) 2.3028  / drink water ✓
epoch : 8 iter : 44 (0m 20s) 2.3054  / drink water ✗ (standing up (from sitting position))
epoch : 17 iter : 31 (0m 40s) 2.3082  / drink water ✗ (brushing teeth)
epoch : 26 iter : 18 (1m 1s) 2.2995  / drink water ✗ (brushing teeth)
epoch : 35 iter : 5 (1m 21s) 2.2987  / drop ✗ (brushing teeth)
epoch : 43 iter : 49 (1m 41s) 2.3019  / drop ✗ (drink water)
epoch : 52 iter : 36 (2m 2s) 2.3060  / drop ✗ (brushing teeth)
epoch : 61 iter : 23 (2m 22s) 2.2970  / drop ✗ (drink water)
epoch : 70 iter : 10 (2m 42s) 2.3071  / drop ✗ (reading)
epoch : 78 iter : 54 (3m 2s) 2.2855  / drop ✗ (standing up (from sitting position))
epoch : 87 iter : 41 (3m 23s) 2.2860  / drop ✗ (brushing teeth)
epoch : 96 iter : 28 (3m 43s) 2.2971  / drop ✗ (tear up paper)
epoch : 105 iter : 15 (4m 4s) 2.3065  / drop ✗ (drink water)
epoch : 114 iter : 2 (4m 24s) 2.3051  / drop ✗ (take off glasses)
epoch : 122 iter : 46 (4m 44s) 2.2941  / drop ✗ (tear up paper)
epoch : 13

4. *Сгенерировать другой датасет с меньшим количеством “кадров” в серии и сравнить улучшилось или ухудшилось качество предсказания. Провести несколько таких итераций, дать свою оценку уменьшению и увеличению кадров, назвать оптимальное, на ваш взгляд, их количество.